In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from table_reader import TableReader
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.model_selection import train_test_split, KFold
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor

Hey folks, I'm just trying out a proof-of-concept jupyter notebook that uses our data retrieval code.

I got sick of working with environment variables so I switched to a new method to store our DB password:
    1. Create a file called config.json in the project root.
    2. Inside, config.json should look like this:
    {
        "database_url":"database_url_goes_here"
    }

TableReader's other vector methods are geodata_vector() and reviews_vector(). Be sure to call close() when you're done so it terminates the connection to the DB.

In [2]:
tr = TableReader()
df = tr.properties_vector(include_amenitites=True)
geodata = tr.geodata_vector()
tr.close()


In [3]:
df = pd.merge(df, geodata, on='listingID', how='left')

In [4]:
features = df[df.columns.drop(['price', 'listingID'])]
label = df['price']

In [60]:
#X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.1)

In [61]:
kf = KFold(n_splits=2)
for train_index, test_index in kf.split(X):
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    mdl = KNeighborsRegressor().fit(X_train, y_train)
score = mdl.score(X_test, y_test)
print(score)

0.14041833534551962


In [45]:
model = ElasticNet()
esfm = SelectFromModel(model)
esfm.fit(features, label)
print(list(features.iloc[:, esfm.get_support(indices=True)]))

['accomodates', 'bathrooms', 'bedrooms', 'beds', 'House', 'Townhouse', 'Entire home/apt', 'Private room', 'Shared room', 'Breakfast', 'Private entrance', 'Carbon monoxide detector', 'Self check-in', 'Indoor fireplace', 'Gym', 'translation missing: en.hosting_amenity_50', 'Suitable for events', 'translation missing: en.hosting_amenity_49', 'Fire extinguisher', 'First aid kit', 'Paid parking off premises', 'Hot water', 'TV', 'Safety card', 'Family/kid friendly', 'Elevator', 'Laptop friendly workspace', 'Free street parking', 'Hangers', 'Columbia Heights, Mt. Pleasant, Pleasant Plains, Park View', 'West End, Foggy Bottom, GWU']


In [46]:
model = Lasso()
sfm = SelectFromModel(model)
sfm.fit(features, label)
print(list(features.iloc[:, sfm.get_support(indices=True)]))

['accomodates', 'bathrooms', 'bedrooms', 'beds', 'Apartment', 'House', 'Townhouse', 'Entire home/apt', 'Shared room', 'Pets live on this property', 'Breakfast', 'Private entrance', 'Carbon monoxide detector', 'Self check-in', 'Cable TV', 'Kitchen', 'Private living room', 'Pool', 'Indoor fireplace', 'Gym', 'Doorman', 'translation missing: en.hosting_amenity_50', 'Heating', 'Internet', 'Suitable for events', 'translation missing: en.hosting_amenity_49', 'Fire extinguisher', 'Lock on bedroom door', 'First aid kit', 'Air conditioning', 'Paid parking off premises', 'Hot water', 'TV', 'Essentials', 'Safety card', 'Family/kid friendly', 'Elevator', 'Laptop friendly workspace', 'Free street parking', 'Buzzer/wireless intercom', 'Pets allowed', 'Hangers', 'zipcode', 'Brightwood Park, Crestwood, Petworth', 'Capitol Hill, Lincoln Park', 'Columbia Heights, Mt. Pleasant, Pleasant Plains, Park View', 'Downtown, Chinatown, Penn Quarters, Mount Vernon Square, North Capitol Street', 'Edgewood, Blooming

In [47]:
model = Ridge()
sfm = SelectFromModel(model)
sfm.fit(features, label)
print(list(features.iloc[:, sfm.get_support(indices=True)]))

['bathrooms', 'bedrooms', 'House', 'Entire home/apt', 'Shared room', 'Breakfast', 'Private entrance', 'Self check-in', 'Pool', 'Gym', 'Doorman', 'Other', 'translation missing: en.hosting_amenity_50', 'Suitable for events', 'Host greets you', 'Paid parking off premises', 'TV', 'Bathtub', 'Family/kid friendly', 'Elevator', 'Free street parking', 'Hangers', 'latitude', 'longitude', 'Capitol Hill, Lincoln Park', 'Cleveland Park, Woodley Park, Massachusetts Avenue Heights, Woodland-Normanstone Terrace', 'Downtown, Chinatown, Penn Quarters, Mount Vernon Square, North Capitol Street', 'Dupont Circle, Connecticut Avenue/K Street', 'Georgetown, Burleith/Hillandale', 'Howard University, Le Droit Park, Cardozo/Shaw', 'Kalorama Heights, Adams Morgan, Lanier Heights', 'Shaw, Logan Circle', 'Southwest Employment Area, Southwest/Waterfront, Fort McNair, Buzzard Point', 'Spring Valley, Palisades, Wesley Heights, Foxhall Crescent, Foxhall Village, Georgetown Reservoir', 'Union Station, Stanton Park, Ki